In [1]:
import pandas as pd
import numpy as np
url = 'https://github.com/mattharrison/datasets/raw/master/data/vehicles.csv.zip'
df = pd.read_csv(url)

/var/folders/61/0_k_4p5x6ps6rb74cyv2c5c80000gn/T/ipykernel_42530/1524119719.py:4: DtypeWarning: Columns (68,70,71,72,73,74,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [2]:
city_mpg = df.city08
highway_mpg = df.highway08

city_mpg, highway_mpg

(0        19
 1         9
 2        23
 3        10
 4        17
          ..
 41139    19
 41140    20
 41141    18
 41142    18
 41143    16
 Name: city08, Length: 41144, dtype: int64,
 0        25
 1        14
 2        33
 3        12
 4        23
          ..
 41139    26
 41140    28
 41141    24
 41142    24
 41143    21
 Name: highway08, Length: 41144, dtype: int64)

<h2>Manipulation Methods</h2>

Manipulation methods help you understand and cleanup the data. They operate on a series and return a new series. These methods usually preserve the index.

<h5>.apply and .where</h5>

The .apply method allows you to apply a function element-wise to every value. 
<p>If you pass in a NumPy function that works on an array, it will broadcast the operation to the series.</p>

It is generally not good to use because it typically operates on every value, so if your data has a million values, it will be called 1 million times.

This breaks the fast, vectorized patterns and falls back to using loop style code.

In [3]:
def gt20(val):
    return val > 20

In [5]:
%%timeit
city_mpg.apply(gt20)

32.1 ms ± 5.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit
city_mpg.gt(20)

589 µs ± 58.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
